# Sentiment Analysis
> * Positive or Negative

In [1]:
import os
from glob import glob
import pandas as pd
import html
import itertools
import pickle
import re

from collections import namedtuple
from gensim.models import doc2vec
import multiprocessing
from konlpy.tag import Mecab
from konlpy.utils import pprint
import numpy as np
from ckonlpy.tag import Twitter as ctwitter
mecab = Mecab()
ct = ctwitter()

C:\Users\pc\Anaconda3\lib\site-packages\gensim\utils.py:860: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")


In [2]:
from sklearn.linear_model import LogisticRegression

## 뉴스 빅데이터 현황으로부터 감정분석 문장 추출

In [3]:
newsSentence = pd.read_csv('./data/sentiment_data/merged_sentiment_data.txt',encoding='utf-8', header=None)
newsSentence.head()

,0,1
0,이젠 민주개혁의 길로 사설 이 90년대가 우리 모두에게 성취의 시대이기를 기원하고...,1
1,민주당과 공화당은 새해부터 정계개편을 적극 추진한다는 방침이며 양당의 김영삼ㆍ김종필...,1
2,"4당,지자제 연합공천 대비 파트너 탐색전 이같은 구상을 실현시키기 위해 민정당은...",1
3,90년대는 국내적으로 정치민주화와 선진국에로의 도약여부가 판가름날 것이고 한반도를 ...,1
4,"특히 노태우대통령이 3일 대국민 특별담화를 발표, 지난시대 의 완전청산을 선언한 후...",1


In [16]:
positiveWords = pd.read_csv('./data/sentiment_data/positive_sentiment_word_from_news.csv',encoding='utf-8',header=None)
negativeWords = pd.read_csv('./data/sentiment_data/negative_sentiment_word_from_news.csv', encoding='utf-8', header=None)

In [17]:
positiveWords = pd.concat([positiveWords, pd.DataFrame(len(positiveWords) * [1], columns=[1])], axis = 1)
negativeWords = pd.concat([negativeWords, pd.DataFrame(len(negativeWords) * [0], columns=[1])], axis = 1)

In [18]:
rawdata = pd.concat([newsSentence, positiveWords, negativeWords])

In [ ]:
rawdata.to_csv('./data/sentiment_data/raw_data_sentiment.txt',index=False, header = False, encoding='utf-8')

## 네이버 영화 리뷰

In [20]:
def read_data(filename):
    with open(filename, 'r',encoding='utf-8') as f:
        data = [line.split('\t') for line in f.read().splitlines()]
        data = data[1:] # header
    return data

In [21]:
rating = read_data('./data/sentiment_data/ratings.txt')

In [22]:
# twitter
def tokenize1(doc):
    return ['/'.join(t) for t in ct.pos(doc)]

In [23]:
# mecab
def tokenize2(doc):
    return ['/'.join(t) for t in mecab.pos(doc)]

## Doc2Vec

In [24]:
from gensim.models.doc2vec import TaggedDocument

In [ ]:
TaggedDocument = namedtuple('TaggedDocument', 'words tags')

In [ ]:
rating_doc_ct = [(tokenize1(row[1]), row[2]) for row in rating]
news_doc_ct = [(tokenize1(newsSentence.loc[idx][0]), newsSentence.loc[idx][1]) for idx in newsSentence.index]
ct_token = rating_doc_ct+news_doc_ct
pickle.dump(ct_token, open('./data/pre_data/pre_data_by_ct_for_sentiment_analysis.pickled','wb'))

In [ ]:
tagged_ct = [TaggedDocument(d, [c]) for d, c in ct_token]
pickle.dump(tagged_ct, open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','wb'))

In [ ]:
tagged_ct = pickle.load(open('./data/pre_data/pre_by_ct_data_tagged_run_docs.pickled','rb'))

In [ ]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(tagged_ct, test_size=0.25, random_state=42)

In [ ]:
doc_vectorizer = doc2vec.Doc2Vec(size=1500, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer.build_vocab(train)

In [ ]:
for epoch in range(30):
    doc_vectorizer.train(train, total_examples=doc_vectorizer.corpus_count, epochs=doc_vectorizer.iter)
    doc_vectorizer.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

In [ ]:
#To save
doc_vectorizer.save('./model/doc2vec_size1500_epoch30_by_ct.model')
pprint(doc_vectorizer.most_similar('문재인/Noun'))
pprint(doc_vectorizer.most_similar('노무현/Noun'))
pprint(doc_vectorizer.most_similar('박근혜/Noun'))

In [ ]:
rating_doc_mecab = [(tokenize2(row[1]), row[2]) for row in rating]
news_doc_mecab = [(tokenize2(newsSentence.loc[idx][0]), newsSentence.loc[idx][1]) for idx in newsSentence.index]
mecab_token = rating_doc_mecab+news_doc_mecab
pickle.dump(mecab_token, open('./data/pre_data/pre_data_by_mecab_for_sentiment_analysis.pickled','wb'))

In [ ]:
tagged_mecab = [TaggedDocument(d, [c]) for d, c in mecab_token]
pickle.dump(tagged_mecab, open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','wb'))

In [ ]:
tagged_mecab = pickle.load(open('./data/pre_data/pre_by_mecab_data_tagged_run_docs.pickled','rb'))

In [ ]:
train2, test2 = train_test_split(tagged_mecab, test_size=0.25, random_state=42)

In [ ]:
doc_vectorizer2 = doc2vec.Doc2Vec(size=1500, alpha=0.025, min_alpha=0.025, seed=1234)
doc_vectorizer2.build_vocab(train2)

In [ ]:
for epoch in range(30):
    doc_vectorizer2.train(train, total_examples=doc_vectorizer2.corpus_count, epochs=doc_vectorizer2.iter)
    doc_vectorizer2.alpha -= 0.002  # decrease the learning rate
    doc_vectorizer2.min_alpha = doc_vectorizer.alpha  # fix the learning rate, no decay

In [ ]:
#To save
doc_vectorizer2.save('./model/doc2vec_size1500_epoch30_by_mecab.model')
pprint(doc_vectorizer2.most_similar('문재인/Noun'))
pprint(doc_vectorizer2.most_similar('노무현/Noun'))
pprint(doc_vectorizer2.most_similar('박근혜/Noun'))